<a href="https://colab.research.google.com/github/prince-musonda/Lulu/blob/main/Lulu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# set up, and install any necessary packages



In [3]:
!pip install -q kokoro>=0.9.2 soundfile flask
!apt-get -qq -y install espeak-ng > /dev/null 2>&1
!pip install -q flask pyngrok


In [4]:
import torch
from PIL import Image
from transformers import AutoProcessor, LlavaForConditionalGeneration
from kokoro import KPipeline
import soundfile as sf
from flask import Flask, request, send_file
from pyngrok import ngrok

# Set up text to speech

In [5]:
def create_speech(text:str,path:str):
    """
    text: input string to convert to audio /speech
    path: path/filename dot file extention to save the file
    """
    speech_generator = KPipeline(lang_code='a')
    generated_speech = speech_generator(text, voice='af_heart')
    for _,_,audio in generated_speech:
      #save generated speech into audio file
      sf.write(path, audio,samplerate=23000)

# Model and model Preprocessor  setup

In [ ]:
model_name = "llava-hf/llava-1.5-7b-hf"
preprocessor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
model = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf",dtype=torch.float16, device_map='auto')

In [9]:

configuration_1 = [
    {
        "role": 'user',
        'content': [
            {"type":"text", "text": """I am a blind person.
            Tell me what you see in a way that is helpful for me. if there is
            any traffic lights tell me what the status is, including whether it's safe to cross the streets or not,
            whether  near a staircase, and warn about Obstacles.
            And start the sentence with the phrase: "I see"
            """},

            {"type": "image"}
        ]
    }
]


def model_inference(image:Image):
  # unified prompt that combines the text with the image
  prompt = preprocessor.apply_chat_template(configuration_1, add_generation_prompt=True)
  input_data = preprocessor(images = image, text=prompt, return_tensors = 'pt').to(device='cuda',dtype=torch.float16)
  output_token = model.generate(**input_data,max_new_tokens = 300)
  output_str = preprocessor.decode(output_token[0],skip_special_tokens = True)
  # clean up output by turn it a list and get the second part of the element list because that
  # corresponds to output that we are interested in
  output_str = output_str.split("ASSISTANT: ")
  output_str = output_str[1]
  return output_str


#Server set up

In [ ]:
app = Flask(__name__)

    # you can uncomment the 2 lines below  and comment the 4th line if you want wan't to use the original url
# ngrok.set_auth_token('36AnchlKkoIZOiSpmc0SfqQa0ac_VRaAXWA1VH2xA6YKdfQx')
# tunnel = ngrok.connect(5000,bind_tls=True,url='mai-unvisceral-finickily.ngrok-free.dev')
    # comment this line below if you choose to uncomment the 2 lines above to use a randomly assigned url
tunnel = ngrok.connect(5000,bind_tls=True)


@app.route("/send_data",methods=["POST"])
def upload_file():
    if request.files:
        user_uploaded_file = request.files.get('file')
        #open upload file as image using the PILLow image class
        image = Image.open(user_uploaded_file)
        # perform inference of the image
        predicted_description = model_inference(image)
        # convert text description into speech
        create_speech(predicted_description,'/content/audio_description.wav')
        # send audio file
        return send_file('/content/audio_description.wav', as_attachment=True)

if __name__ == "__main__":
  print("please, connet to this address in your Lulu  device: "+tunnel.public_url)
  app.run(port=5000, use_reloader=False)
